In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt

The potential energy between two bodies of masses $M$ and $m$ a distance $r$ apart is
$$\phi(r) = -\frac{GMm}{r}$$
where $G$ is Newton's gravitational constant.
The force between them is
$$F = -\nabla\phi = -\frac{GMm\,\mathbf{r}}{r^3}.$$
We'll work in a unit system where $GMm = 1$ because I forget what the mass of the sun is and we don't have internet up here.

In [ ]:
def potential_energy(x):
    r = np.sqrt(np.dot(x, x))
    return -1 / r

def force(x):
    r = np.sqrt(np.dot(x, x))
    return - x / r**3

We'll start the planet at the position $(1, 0)$ and assume its initial velocity is $(0, 1)$.
Later we can add a bit of eccentricity.

In [ ]:
x_0 = np.array([1.0, 0.0])
p_0 = np.array([0.0, 1.0])

Our goal is to solve the ODE
$$\begin{align}
\frac{d}{dt}\mathbf{x} & = \mathbf{p} \\
\frac{d}{dt}\mathbf{p} & = F(\mathbf{x}) = -\frac{\mathbf{x}}{|x|^3}
\end{align}$$
subject to the initial conditions $x(t = 0) = x_0$, $p(t = 0) = p_0$.
While this problem is exactly solvable, most ODEs aren't.
So we can try out numerical methods on this problem to get a better feel for how well they work.
Numerical methods for ODEs are based on approximating time derivatives by a difference quotient, and in order to do that, we have to pick a timestep.
Before we go on to actually writing the method, let's pick our timestep and total number of steps.

In [ ]:
dt = 1.0 / 24
num_steps = 2400

Then we'll create some arrays to store the solution and initialize it.

In [ ]:
xs = np.zeros((num_steps + 1, 2))
ps = np.zeros((num_steps + 1, 2))

xs[0] = x_0
ps[0] = p_0

Everyone's first numerical method is the *first-order*, *explicit* scheme:
$$\begin{align}
\frac{x_{t + 1} - x_t}{\delta t} & = p_t \\
 & \\
\frac{p_{t + 1} - p_t}{\delta t} & = F(x_t)
\end{align}$$

In [ ]:
for t in range(num_steps):
    xs[t + 1] = xs[t] + dt * ps[t]
    ps[t + 1] = ps[t] + dt * force(xs[t])

Now let's see what the trajectory looks like:

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.plot(xs[:, 0], xs[:, 1]);

Seems a little off maybe, but how?
A pretty important property of classical mechanical systems is that they conserve energy -- the quantity
$$E = \frac{|p|^2}{2} - \frac{1}{|x|}$$
is constant along trajectories.
How does the first-order explicit scheme do?

In [ ]:
E = 0.5 * np.sum(ps**2, axis=1) - 1.0 / np.sqrt(np.sum(xs**2, axis=1))

fig, ax = plt.subplots()
ax.set_xlabel("timestep")
ax.set_ylabel("energy")
ax.plot(E);

The system seems to be gaining energy in time, which is clearly wrong!
We could take a smaller timestep $\delta t$ and more steps to mitigate this effect, or we could see if there are nicer methods.

Earlier, we updated the force based on the old value of the position, but by that point we had already computed a new position.
What if we instead tried this:
$$\begin{align}
\frac{x_{t + 1} - x_t}{\delta t} & = p_t \\
 & \\
\frac{p_{t + 1} - p_t}{\delta t} & = F(x_{t + 1})
\end{align}$$
where we use the newly-computed value $x_{t + 1}$ to compute the force.

In [ ]:
xs = np.zeros((num_steps + 1, 2))
ps = np.zeros((num_steps + 1, 2))

xs[0] = x_0
ps[0] = p_0

for t in range(num_steps):
    xs[t + 1] = xs[t] + dt * ps[t]
    ps[t + 1] = ps[t] + dt * force(xs[t + 1])

The trajectories look more circular and they stay bounded!

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.plot(xs[:, 0], xs[:, 1]);

The energy isn't conserved exactly, but it stays bounded too!

In [ ]:
E = 0.5 * np.sum(ps**2, axis=1) - 1.0 / np.sqrt(np.sum(xs**2, axis=1))

fig, ax = plt.subplots()
ax.set_xlabel("timestep")
ax.set_ylabel("energy")
ax.plot(E);